In [15]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO

In [63]:
year = 2000

In [64]:
URL = f'https://www.basketball-reference.com/leagues/NBA_{year}_totals.html#totals_stats::pts'

site = requests.get(URL)
soup = BeautifulSoup(site.text, 'html')
table = soup.find('table')
tbody = table.find('tbody')
playername = tbody.find_all('td', {'data-stat':'player'})
realplayernames = [name.text for name in playername]
realplayernames = list(dict.fromkeys(realplayernames)) #Removendo nomes duplicados
print(realplayernames)

['Tariq Abdul-Wahad', 'Shareef Abdur-Rahim', 'Cory Alexander', 'Ray Allen*', 'Rafer Alston', 'John Amaechi', 'Derek Anderson', 'Kenny Anderson', 'Nick Anderson', 'Shandon Anderson', 'Chris Anstey', 'Greg Anthony', 'B.J. Armstrong', 'Darrell Armstrong', 'Chucky Atkins', 'Stacey Augmon', 'Isaac Austin', 'Anthony Avent', 'William Avery', 'Toby Bailey', 'Vin Baker', 'Charles Barkley*', 'Dana Barros', 'Brent Barry', 'Drew Barry', 'Jon Barry', 'Tony Battie', 'Jonathan Bender', 'Benoit Benjamin', 'Corey Benjamin', 'Mario Bennett', 'Travis Best', 'Mike Bibby', 'Chauncey Billups', 'Mookie Blaylock', 'Corie Blount', 'Muggsy Bogues', 'Etdrick Bohannon', 'Calvin Booth', 'Lazaro Borrell', 'Cal Bowdler', 'Bruce Bowen', 'Ryan Bowen', 'Ira Bowman', 'Earl Boykins', 'Shawn Bradley', 'A.J. Bramlett', 'Elton Brand', 'Terrell Brandon', 'Chucky Brown', 'Dee Brown', 'Marcus Brown', 'P.J. Brown', 'Randy Brown', 'Rick Brunson', 'Kobe Bryant*', 'Mark Bryant', 'Greg Buckner', 'Jud Buechler', 'Rodney Buford', 'Ma

In [17]:
tr = tbody.find_all('tr')
urlname = []

for i in tr:
    td = i.find('td')
    if td is not None:
        urlname.append(list(td.attrs.values())[1])

urlname = list(dict.fromkeys(urlname))

In [18]:
first_letter = []
for i in realplayernames:
    first_letter.append(i[:1])

In [47]:
from time import sleep

headers = list(range(1,83))
headers.insert(0,'Player')
dff = pd.DataFrame(columns = headers)

ignore_rows = [20,41,62,83]

for x in range(0,len(realplayernames)):
    URL = f'https://www.basketball-reference.com/players/{first_letter[x]}/{urlname[x]}/gamelog/{year}'
    sleep(5)
    site = requests.get(URL)
    soup = BeautifulSoup(site.text,'html')
    table = soup.find('table',{'id':'pgl_basic'})
    tbody = table.find('tbody')
    tr = tbody.find_all('tr')

    points = []
    for k in range(0,len(tr)):
        points_value = tr[k].find('td',{'data-stat':'pts'})
        if points_value is not None: 
            points.append(int(points_value.text))
        elif k not in ignore_rows:
            points.append(0)
    if len(points) < 82:
        for i in range(0,(83 - len(points))):
            points.append(0)
    new_row = points[0:82]
    new_row.insert(0,realplayernames[x])
    new_df = pd.DataFrame([new_row], columns = dff.columns)
    dff = pd.concat([dff,new_df], ignore_index = True)


In [54]:
for x in range(2,83):
    dff[x] = dff[[x-1,x]].sum(axis=1)

In [56]:
dff = dff.sort_values(by=82,ascending=False)

In [58]:
dff.to_csv(f'NBA {year - 1}-{year[2:]} Regular Season Scoring Leaders.csv', index=False)